[붓꽃 품종 분류: 3개의 붓꽃 품종 분류]

- ds: 내장 ds
- feature: 4개
- target: 1개의 품종
- 학습: 지도학습 중 분류

1) 데이터 준비

In [115]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [116]:
data=load_iris(as_frame=True)

In [117]:
#bunch 인스턴스: dict와 유사
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [118]:
featuredf=data['data']
targetsr=data['target']

In [119]:
featuredf.shape, targetsr.shape

((150, 4), (150,))

In [120]:
featuredf.head(1), targetsr.head(1)

(   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 0                5.1               3.5                1.4               0.2,
 0    0
 Name: target, dtype: int32)

2) 학습을 위한 DS 준비
- 학습용, 검증용, 테스트용

In [121]:
#학습용/테스트용 먼저 분리
x_train, x_test, y_train, y_test=train_test_split(featuredf, targetsr, stratify=targetsr)

In [122]:
#학습용 -> 학습용/검증용 분리
x_train,x_val,y_train,y_val=train_test_split(x_train, y_train, stratify=y_train)

In [123]:
print(f'train ds: {x_train.shape[0]} {x_train.shape[0]/featuredf.shape[0]}%')
print(f'val ds: {x_val.shape[0]} {x_val.shape[0]/featuredf.shape[0]:.2f}%')
print(f'test ds: {x_test.shape[0]} {x_test.shape[0]/featuredf.shape[0]:.2f}%')

train ds: 84 0.56%
val ds: 28 0.19%
test ds: 38 0.25%


3) 교차검증 방식: 검증용 데이터에 최적화 되는 것을 방지하고자

In [124]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [125]:
dtc_model=DecisionTreeClassifier()
accuracys=[]
kfold=KFold()  #기본 k=5

In [126]:
for idx,(train_index, val_index) in enumerate(kfold.split(featuredf),1):   # <-학습용/검증용 ds 인덱스 반환
    print(f'train_index: {train_index.tolist()}')

    x_train, y_train=featuredf.iloc[train_index.tolist()],targetsr[train_index.tolist()]
    x_val,y_val=featuredf.iloc[val_index.tolist()],targetsr[val_index.tolist()]

    dtc_model.fit(x_train,y_train)

    t_accuracy=dtc_model.score(x_train,y_train)
    v_accuracy=dtc_model.score(x_val,y_val)

    accuracys.append([t_accuracy,v_accuracy])
    print(f'[{idx}번째 정확도] train: {t_accuracy}, val: {v_accuracy}')

train_index: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149]
[1번째 정확도] train: 1.0, val: 1.0
train_index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 1

In [127]:
accuracys

[[1.0, 1.0], [1.0, 1.0], [1.0, 0.9], [1.0, 0.9333333333333333], [1.0, 0.8]]

In [128]:
train_mean=sum([value[0] for value in accuracys])/kfold.n_splits
test_mean=sum([value[1] for value in accuracys])/kfold.n_splits

print(f'train 정확도: {train_mean}  val 정확도:{test_mean:.2f}')

train 정확도: 1.0  val 정확도:0.93


In [129]:
accuracys=[]

skfold=StratifiedKFold()

for idx,(train_index, val_index) in enumerate(skfold.split(featuredf,targetsr),1):   # <-target(정답/레이블)의 비율을 고려해서 데이터 나눔
    #print(f'train_index: {train_index.tolist()}')

    x_train, y_train=featuredf.iloc[train_index.tolist()],targetsr[train_index.tolist()]
    x_val,y_val=featuredf.iloc[val_index.tolist()],targetsr[val_index.tolist()]

    dtc_model.fit(x_train,y_train)

    t_accuracy=dtc_model.score(x_train,y_train)
    v_accuracy=dtc_model.score(x_val,y_val)

    accuracys.append([t_accuracy,v_accuracy])
    print(f'[{idx}번째 정확도] train: {t_accuracy}, val: {v_accuracy}')

[1번째 정확도] train: 1.0, val: 0.9666666666666667
[2번째 정확도] train: 1.0, val: 0.9666666666666667
[3번째 정확도] train: 1.0, val: 0.9
[4번째 정확도] train: 1.0, val: 1.0
[5번째 정확도] train: 1.0, val: 1.0


In [130]:
train_mean=sum([value[0] for value in accuracys])/skfold.n_splits
test_mean=sum([value[1] for value in accuracys])/skfold.n_splits

print(f'train 정확도: {train_mean}  val 정확도:{test_mean:.2f}')

train 정확도: 1.0  val 정확도:0.97


[교차검증 및 성능평가 동시 진행 함수]
- cross_val_score, cross_val_predict
- cross_validate

In [131]:
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate

[1] ds -> 학습용/테스트용 분리

In [148]:
x_train,x_test,y_train,y_test=train_test_split(featuredf,targetsr,stratify=targetsr)

[2] 검증

In [149]:
#cross_val_predict
predict=cross_val_predict(dtc_model,featuredf,targetsr, cv=3)

In [150]:
print(f'predict: {predict}')

predict: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [151]:
#cross_val_score
cross_val_score(dtc_model,featuredf,targetsr)

array([0.96666667, 0.96666667, 0.9       , 0.93333333, 1.        ])

In [161]:
#cross_validate: cv 수가 늘어날수록 검증용 데이터 작아짐
result=cross_validate(dtc_model,featuredf,targetsr,return_train_score=True,return_estimator=True, cv=10)

In [162]:
result

{'fit_time': array([0.00321555, 0.00299931, 0.00300002, 0.00099945, 0.00099993,
        0.00200105, 0.00199986, 0.00099993, 0.00100017, 0.00199986]),
 'score_time': array([0.00101423, 0.00200009, 0.00099969, 0.00100017, 0.00100017,
        0.00100183, 0.00100017, 0.00100017, 0.00099993, 0.00099993]),
 'estimator': [DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier(),
  DecisionTreeClassifier()],
 'test_score': array([1.        , 0.93333333, 1.        , 0.93333333, 0.93333333,
        0.86666667, 0.93333333, 1.        , 1.        , 1.        ]),
 'train_score': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [163]:
resultdf=pd.DataFrame(result)
resultdf

,fit_time,score_time,estimator,test_score,train_score
0,0.003216,0.001014,DecisionTreeClassifier(),1.000000,1.0
1,0.002999,0.002000,DecisionTreeClassifier(),0.933333,1.0
2,0.003000,0.001000,DecisionTreeClassifier(),1.000000,1.0
3,0.000999,0.001000,DecisionTreeClassifier(),0.933333,1.0
4,0.001000,0.001000,DecisionTreeClassifier(),0.933333,1.0
5,0.002001,0.001002,DecisionTreeClassifier(),0.866667,1.0
6,0.002000,0.001000,DecisionTreeClassifier(),0.933333,1.0
7,0.001000,0.001000,DecisionTreeClassifier(),1.000000,1.0
8,0.001000,0.001000,DecisionTreeClassifier(),1.000000,1.0
9,0.002000,0.001000,DecisionTreeClassifier(),1.000000,1.0


In [164]:
resultdf=pd.DataFrame(result).loc[:,['test_score','train_score']]
resultdf

,test_score,train_score
0,1.000000,1.0
1,0.933333,1.0
2,1.000000,1.0
3,0.933333,1.0
4,0.933333,1.0
5,0.866667,1.0
6,0.933333,1.0
7,1.000000,1.0
8,1.000000,1.0
9,1.000000,1.0


[3] 최적화된 모델 추출

In [144]:
best_model=result['estimator'][4]

In [145]:
best_model

DecisionTreeClassifier()

[4] test 데이터로 확인

In [146]:
best_model.predict(x_test)

array([0, 2, 0, 1, 0, 1, 2, 1, 1, 1, 2, 2, 1, 0, 2, 1, 2, 2, 0, 0, 2, 0,
       2, 0, 0, 1, 1, 2, 2, 2, 1, 0, 2, 0, 0, 1, 0, 1])

In [147]:
best_model.score(x_test,y_test)

1.0